In [1]:
##########==========##########==========##########==========##########==========

## HEAD

#### HEAD 01 - toggle user settings

In [2]:
## determine whether to cache data from some time consuming tasks
## selecting True for any option increases run time and memory usage
## selecting True for all options fully executes the code from start-to-finish
settings = {
    'num_parallel_cores': 13,
    'full_data_mode': True, ## model all data, not just a sample for code dev
    
    'collect_data': True, ## toggles twitter api pulls in PULL01-03
    
    'rebuild_word_data': True, ## rebuild vs cache load for word_data MUNG02
    'rebuild_tweet_words': True, ## rebuild vs cache load tweet_words MUNG03
    'rebuild_user_token': True, ## rebuild v cache load user_token_tally MUNG04
    
    'rebuild_logistic_hparams': True, ## rebuild v cache ml training MODE04
    'rebuild_bayes_hparams': True, ## rebuild v cache ml training MODE05
    'rebuild_forest_hparams': True, ## rebuild v cache ml training MODE06
    'rebuild_adaboost_hparams': True ## rebuild v cache ml training MODE07
    }

#### HEAD02 - load libraries

In [3]:
import tweepy
import numpy  as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
from time import sleep
from os.path import exists
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

#### HEAD03 - load data files

In [4]:
## read in roster of handles
user_data = pd.read_excel("A_Input/twitter_handles.xlsx")

## drop all but a handful of cases if in test mode
if not settings['full_data_mode']:
    user_data = user_data.sample(125, random_state = 5542)

## read in twitter credentials; initialize api connection+
twitter_credentials = pd.read_csv('../api_keys/twitter.csv').set_index('item')
twitter_credentials = tweepy.OAuth1UserHandler(
    consumer_key = twitter_credentials.loc['API Key', 'string'],
    consumer_secret = twitter_credentials.loc['API Key Secret', 'string'],
    access_token = twitter_credentials.loc['Access Token', 'string'],
   access_token_secret = twitter_credentials.loc['Access Token Secret', 'string']
    )
api = tweepy.API(twitter_credentials)

/Users/s8/opt/anaconda3/envs/py310/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


#### HEAD04 - create build or cache decision function

In [5]:
## build switching function to execute code or cache results
def build_or_cache_csv(address, function, build_bool):
    if build_bool or not exists(address):
        x = function()
        x.to_csv(address, index = False)
        return x
    else:
        return pd.read_csv(address)
    
def build_or_cache_pickle(address, function, build_bool):
    if build_bool or not exists(address):
        x = function()
        conn = open(address, 'wb')
        pickle.dump(x, conn)
        conn.close()
        return x
    else:
        conn = open(address, 'rb')
        x = pickle.load(conn)
        conn.close()
        return x

In [6]:
##########==========##########==========##########==========##########==========

## HAND – Gather Twitter handles for test accounts

#### HAND01 - extract handles from roster URLs

In [7]:
## extract handles from roster urls
user_data['handle'] = user_data.url.str.replace('https://twitter.com/', '',
            regex = False).str.strip().str.lower()

In [8]:
##########==========##########==========##########==========##########==========

## PULL - Pull Twitter data from the API

#### PULL01 - query API for each roster handle's user_timeline data

In [9]:
## extract tweet data from api object
def refine_tweet_data(x):
    tweet_data = list()
    for i in range(0, len(x)):
        tweet_data.append({
            'tweet_id': x[i].id, 
            'created_at': x[i].created_at, 
            'lang': x[i].lang,
            'full_text': x[i].full_text,
            'screen_name': x[i].author.screen_name,
            'verified' : x[i].author.verified
        })
    return pd.DataFrame(tweet_data)

## define function to pull user tweet data and apply function to extract tweet data
def pull_tweet_data(handles = user_data.handle, a = api):
    tweet_data = list()
    for i in handles:
        try:
            user_tweets = a.user_timeline(
                screen_name = i, count = 200, tweet_mode = 'extended', 
                exclude_replies = True, include_rts = False)
            tweet_data.append(refine_tweet_data(user_tweets))
            sleep(1)
        except:
            pass
    tweet_data = pd.concat(tweet_data)
    tweet_data['screen_name'] = tweet_data['screen_name'].str.lower()
    return tweet_data

## execute code
if settings['collect_data']:
    tweet_data = pull_tweet_data()

#### PULL02 - tabulate tweet statistics, divide users into train/tune/test sets

In [10]:
## combile tweet count and verification status at the user level
def enhance_user_data(td, ud = user_data):
    
    def np_size(x): return x.size
    
    ## calculate tweet summary statistics
    td_original = tweet_data
    td = td.copy()
    verified = td.groupby('screen_name').mean()
    tweets = td['screen_name'].value_counts()
    td = pd.concat([verified, tweets], axis = 1).reset_index()
    td.columns = ['handle', 'verified', 'tweets']
    
    ## merge statistics into the user_data object
    ud = pd.merge(ud, td, on = 'handle', how = 'left')
    td_original = td_original.drop(['verified'], axis = 1)
    ud = ud.drop(['url'], axis = 1).reset_index(drop = True)
    ud = ud.fillna({'tweets': 0}).astype({'tweets': int})
    
    ## divide users into train, and test subsets
    ml_set = pd.Series(['train', 'test'], name = 'ml_set').sample(
                n = ud.shape[0], replace = True, weights = [0.8, 0.20],
                random_state = 2006)
    ud['ml_set'] = ml_set.values
    ud.loc[ud.tweets == 0, 'ml_set'] = 'exclude'
    
    return ud, td_original

## execute code
if settings['collect_data']:
    user_data, tweet_data = enhance_user_data(
        tweet_data[['screen_name', 'verified']])

#### PULL03 - save datasets to disk

In [11]:
## save user/tweet datasets to disk as csvs
if settings['collect_data']:
    user_data.to_csv('B_Process/user_data.csv', index = False)
    tweet_data.to_csv('B_Process/tweet_data.csv', index = False)
else:
    user_data = pd.read_csv('B_Process/user_data.csv')
    tweet_data = pd.read_csv('B_Process/tweet_data.csv')

In [12]:
##########==========##########==========##########==========##########==========

## MUNG - Process Twitter data to model-ready format

#### MUNG01 - parse tweet text into tokens

In [13]:
## tokenize, remove capitalization, and remove duplicate tokens
def nlp_tokenize_tweet(x):
    x = x.lower()
    x = word_tokenize(x)
    x = list(set(x))
    return x

## execute code
tweet_data['tokens'] = tweet_data.full_text.apply(nlp_tokenize_tweet)

#### MUNG02 - create word/token level dataset

In [14]:
## create word/token level dataset and identify valid word tokens
def make_word_data(td = tweet_data):

    ## flatten token lists and count occurances
    word_data = list()
    for i in td.tokens:
        word_data += i
    word_data = pd.Series(word_data, name = 'count').value_counts()
    word_data = word_data.sort_values(ascending = False)
    word_data = pd.DataFrame(word_data)
    
    ## determine which tokens occur often enough to warrant inclusion
    word_data['valid'] = word_data['count'] > max(
        word_data['count'].quantile(0.2), 3)
    word_data['word'] = word_data.index
    
    ## determine part of speech for eligible tokens
    speech_part = word_data['word'].loc[word_data['valid']].values
    speech_part = pos_tag(speech_part)
    speech_part = [i[1][0].lower() for i in speech_part]
    word_data['pos'] = '.'
    word_data.loc[word_data['valid'], 'pos'] = speech_part
    
    ## lemmatize
    WNL = WordNetLemmatizer()
    word_data['token'] = None
    for i in word_data.word:
        if not word_data.loc[i, 'valid']: 
            break
        if word_data.loc[i, 'pos'] in 'abcdefghijklmnopqrstuvwxyz':
            try:
                word_data.loc[i, 'token'] = WNL.lemmatize(
                    word_data.loc[i, 'word'],
                    pos = word_data.loc[i, 'pos']
                )
            except:
                word_data.loc[i, 'token'] = word_data.loc[i, 'word']
        else:
            word_data.loc[i, 'valid'] = False
        
    return word_data.reset_index(drop = True)

## execute code
word_data = build_or_cache_csv(
    address = 'B_Process/word_data.csv',
    function = make_word_data,
    build_bool = settings['rebuild_word_data']
    )

#### MUNG03 - generate a tokens x tweets link database

In [15]:
def make_tweet_token_data(td = tweet_data, wd = word_data):
    
    wd = wd.set_index('word')
    
    ## replicate tweet ids
    n = td.tokens.apply(len).values
    tweet_tokens = pd.Series(np.repeat(td.tweet_id.values, n), name = "tweet_id")
    tweet_tokens = pd.DataFrame(tweet_tokens)
    
    ## allocate words to the new dataset
    words = list()
    for i in td.tokens:
        words += i
    tweet_tokens['words'] = words
    
    ## convert words to tokens
    tweet_tokens['tokens'] = wd.loc[
        tweet_tokens.words.values, 'token'].values
    tweet_tokens = tweet_tokens.dropna()

    return tweet_tokens.reset_index(drop = True)

## execute code
tweet_words = build_or_cache_csv(
    address = 'B_Process/tweet_words.csv',
    function = make_tweet_token_data,
    build_bool = settings['rebuild_tweet_words']
    )
tweet_data = tweet_data.drop('tokens', axis = 1)

#### MUNG04 - generate a tokens x users count; drop tokens with only one user

In [16]:
def make_user_token_matrix(td = tweet_data, tw = tweet_words, ud = user_data):
    
    ## count of number of times each user wrote each token
    tw = tw.merge(right = td[['screen_name', 'tweet_id']],
                  how = 'left', on = 'tweet_id')
    tw = tw.drop(['tweet_id', 'words'], axis = 1).groupby('screen_name')
    tw = tw.value_counts()
    tw.name = 'count'
    tw = tw.reset_index().set_index('screen_name')
    tw = tw.pivot(columns = 'tokens').fillna(0).astype(int)
    tw = tw.droplevel(axis = 1, level = 0)
    
    ## remove tokens that fewer than 3 or more than 80 percent of users use
    valid_usage = (tw > 0).astype(int).sum().values
    valid_usage = (valid_usage > 2
                  ) & (valid_usage < int(tw.shape[0] * 0.8))
    tw = tw.loc[:, valid_usage]
    
    ## standardize matrix as words per 1,000 tweets
    denom = pd.DataFrame({'handle': tw.index}).merge(
        ud[['handle', 'tweets']],
        how = 'left', on = 'handle'
        ).set_index('handle').squeeze().fillna(1)
    denom.loc[denom < 1] = 1
    tw = (tw.divide(denom, axis = 0) * 1000).astype(int)
    tw = tw.reset_index().rename({'level_0':'screen_name'}, axis = 1)
    
    return tw


## execute code
user_token_matrix = build_or_cache_csv(
    address = 'B_Process/user_token_matrix.csv',
    function = make_user_token_matrix,
    build_bool = settings['rebuild_user_token']
    ).set_index('screen_name')

In [17]:
##########==========##########==========##########==========##########==========

## HYPE – Train models and tune hyperparameters

#### HYPE01 - build x/y train/test datasets

In [18]:
## unpack train and test datasets
def split_xy_data(ml_cat):
    i = user_data.loc[user_data.ml_set == ml_cat, 'handle'].values
    
    x = user_token_matrix.loc[i, :]
    
    y1 = user_data.set_index('handle').loc[i, 'group'] == 'USA House'
    y1 = y1.astype(int)
    
    y2 = user_data.set_index('handle').loc[i, 'party'] == 'Republican'
    y2 = y2.astype(int)
    
    return x, y1, y2

## execute code
train_x, train_y1, train_y2 = split_xy_data('train')
test_x, test_y1, test_y2    = split_xy_data('test')

In [19]:
## delete objects to clear up memory in prep for modeling
del tweet_words, tweet_data, user_token_matrix, word_data

#### HYPE02 - build pca versions of x dataset

In [20]:
## fit pca model in order to simplify and enhance feature matrix
model_pca = PCA().fit(train_x)

## generate pca transformations of all feature matrices
train_x_pca = model_pca.transform(train_x)
test_x_pca  = model_pca.transform(test_x)

#### HYPE03 - formulate generic function for hyperparameter search

In [21]:
## find best hyperparameters
def find_hparams():
    model1 = the_model.fit(X = train_x, y = train_y1)
    model2 = the_model.fit(X = train_x, y = train_y2, sample_weight = train_y1)
    model3 = the_model.fit(X = train_x_pca, y = train_y1)
    model4 = the_model.fit(
        X = train_x_pca, y = train_y2, sample_weight = train_y1)
    return {'feature_stage1': model1, 'feature_stage2': model2,
            'pca_stage1': model3, 'pca_stage2': model4}


#### HYPE04 - find best logistic regression model parameters

In [22]:
## formulate model
the_model = GridSearchCV( 
    estimator = LogisticRegression(penalty = 'l1',
                                   class_weight = 'balanced',
                                   solver = 'saga',
                                   max_iter = 2**10
                                  ),
    scoring = 'f1',
    cv = 5, 
    n_jobs = settings['num_parallel_cores'], 
    param_grid = {'C': [2. ** i for i in range(-2, 6)]}
)

## execute code
logistic_hparams = build_or_cache_pickle(
    address = 'B_Process/models/logistic_hparams.pickle',
    function = find_hparams,
    build_bool = settings['rebuild_logistic_hparams']
    )

#### HYPE05 - find best naive bayes model parameters

In [23]:
## formulate model
the_model = GridSearchCV( 
    estimator = GaussianNB(),
    scoring = 'f1',
    cv = 5, 
    n_jobs = settings['num_parallel_cores'], 
    param_grid = {'priors': [
        (0.5, 0.5), 
        (1 - train_y1.mean(), train_y1.mean()),
        (1 - train_y2.mean(), train_y2.mean()),
    ]}
)

## execute code
bayes_hparams = build_or_cache_pickle(
    address = 'B_Process/models/bayes_hparams.pickle',
    function = find_hparams,
    build_bool = settings['rebuild_bayes_hparams']
    )

#### HYPE06 - find best random forest model parameters

In [24]:
## formulate model
the_model = GridSearchCV( 
    estimator = RandomForestClassifier(class_weight = 'balanced'),
    scoring = 'f1',
    cv = 5, 
    n_jobs = settings['num_parallel_cores'], 
    param_grid = {
        'n_estimators': [2**i for i in range(6, 10)],
        'min_samples_leaf': [2**i for i in range(2, 5)]
    }
)

## execute code
forest_hparams = build_or_cache_pickle(
    address = 'B_Process/models/forest_hparams.pickle',
    function = find_hparams,
    build_bool = settings['rebuild_forest_hparams']
    )

#### HYPE07 - find best AdaBoost model parameters and train model

In [27]:
## formulate model
the_model = GridSearchCV( 
    estimator = AdaBoostClassifier(base_estimator = None),
    scoring = 'f1',
    cv = 5, 
    n_jobs = settings['num_parallel_cores'], 
    param_grid = {
        'n_estimators':  [int(2**i) for i in range(6, 10)],
        'learning_rate': [2**i for i in range(-2, 3)]
    }
)

## execute code
adaboost_hparams = build_or_cache_pickle(
    address = 'B_Process/models/adaboost_hparams.pickle',
    function = find_hparams,
    build_bool = settings['rebuild_adaboost_hparams']
    )

In [ ]:
##########==========##########==========##########==========##########==========

## EVAL - Analyse model performance and interpret

#### EVAL01 - compile hyperparameter search statistics

#### EVAL02 - compile best model performance statistics

#### EVAL03 - calculate comparison performance stats for random guess model

#### EVAL04 - determine variable importance

#### EVAL05 - score tweets based on variable importance

#### EVAL06 - extract softmax scores

In [ ]:
##########==========##########==========##########==========##########==========

## FOOT - display objects as needed